# User interface and report generator

## Context

In this project, I will use the ipywidgets library to create an user interface and a report generator. The user will be able to filter a table, do predefined calculations and graphs using the filtered table, save the data created and generate a report. The user will also be able to include comments on said report.

## Import libraries

In [1]:
import pandas as pd
import numpy as np
import datetime
import ipywidgets as widgets #https://ipywidgets.readthedocs.io/en/stable/examples/Widget%20List.html
#https://towardsdatascience.com/bring-your-jupyter-notebook-to-life-with-interactive-widgets-bc12e03f0916
from ipywidgets import interact, interactive
from IPython.display import display, clear_output
import plotly.express as px

import io
import functools

## Data Analysis

### Table functions

In [2]:
def get_group_mean(lst_group):
    """
    Calculate aggregated mean of global variable df.

    Parameters
    ----------

    lst_group : list
        List containing columns to aggregate

    Returns
    -------
    mean : DataFrame
        Dataframe showing aggregated mean

    See Also
    --------
    get_group_sum : Calculate aggregated sum
    get_group_count : Calculate aggregated count
    """    
    global df
    mean = df.groupby(lst_group).agg("mean")
    return mean

def get_group_sum(lst_group):
    """
    Calculate aggregated sum of global variable df

    Parameters
    ----------

    lst_group : list
        List containing columns to aggregate

    Returns
    -------
    sums : DataFrame
        Dataframe showing aggregated sum

    See Also
    --------
    get_group_mean : Calculate aggregated mean
    get_group_count : Calculate aggregated count
    """   
    global df
    sums = df.groupby(lst_group).agg("sum")
    return sums

def get_group_count(lst_group):
    """
    Calculate aggregated count of global variable df.

    Parameters
    ----------
    
    lst_group : list
        List containing columns to aggregate

    Returns
    -------
    count : DataFrame
        Dataframe showing aggregated count

    See Also
    --------
    get_group_mean : Calculate aggregated mean
    get_group_sum : Calculate aggregated sum
    """
    global df
    count = df.groupby(lst_group).agg("count")
    return count

def do_describe(nothing):
    """
    Do describe Pandas function to df.

    Parameters
    ----------
    nothing : Any datatype
        Placeholder variable
        
    Returns
    -------
    df.describe() : DataFrame
        Dataframe showing the result of describe() Pandas function

    See Also
    --------
    get_group_mean : Calculate aggregated mean
    get_group_sum : Calculate aggregated sum
    get_group_sum : Calculate aggregated count
    """
    global df
    return df.describe()

### Plot functions

In [3]:
def do_scatter (x, y, color):
    global df
    return px.scatter(df, x=x, y=y, color = color)

def do_bar (x, y):
    global df
    return px.bar(df, x=x, y=y)

### Actions

In [4]:
dict_functions = {"Mean ({})".format('smoker') : [get_group_mean, ['smoker']],
                  "Sum ({})".format('age') : [get_group_sum, ['age']],
                  "Count ({})".format('region') : [get_group_count, ['region']],
                  "Describe" : [do_describe, ""]}

dict_plots = {"Plot ({} vs {} vs {})".format('age', 'bmi', 'sex'): [do_scatter, 'age', 'bmi', 'sex'], 
              "Plot ({} vs {} vs {})".format('smoker', 'charges', 'sex'): [do_scatter, 'smoker', 'charges', 'sex'],
              "Plot ({} vs {})".format('region', 'charges'): [do_bar, 'region', 'charges']}

## Generic widget functions

In [5]:
def create_uploader (accept_multiple, extensions):
    """
    Create file uploader widget.

    It uses the ipywidgets (as widgets) library to create a FileUpload instance.

    Parameters
    ----------
    accept_multiple : boolean
        True to accept multiple files; False to accept a single file.

    extensions : str
        File extension accepted; if multiple, use '.extension_1, .extension_2'.

    Returns
    -------
    uploader : ipywidget.FileUpload instance
        File uploader widget

    See Also
    --------
    join_uploader : Joins functions create_uploader and create_button
    """
    uploader = widgets.FileUpload(accept=extensions, multiple=accept_multiple)
    return uploader


def create_button(text):
    """
    Create button widget.

    It uses the ipywidget as widgets library to create a button instance.

    Parameters
    ----------
    text : str
        Description to include on the button.

    Returns
    -------
    ipywidget.Button instance
        Button widget

    See Also
    --------
    join_uploader : Joins functions create_uploader and create_button
    """
    return widgets.Button(description=text)

def join_uploader(text, accept_multiple, extensions='.csv'):
    """
    Joins functions create_uploader and create_button.

    Puts the widgets FileUpload and Button in a box

    Parameters
    ----------
    text : str
        Description to include on the button.

    accept_multiple : boolean
        True to accept multiple files; False to accept a single file.

    extensions : str, default '.csv'
        File extension accepted; if multiple, use '.extension_1, .extension_2'.

    Returns
    -------
    ipywidget.Box instance
        Box widget containing an uploader and button widgets

    See Also
    --------
    create_uploader : Create file uploader widget
    create_button : Create button widget
    """
    uploader = create_uploader (accept_multiple, extensions)
    button = create_button(text)
    return widgets.HBox([uploader, button]), uploader, button

def do_files_to_dfs (uploader):
    """
    Transforms the contents uploaded to individual dataframes.
    Each file should have its dataframe.
    The ipywidgets.FileUpload() does not save filepaths.
    
    Parameters
    ----------
    uploader : ipywidgets.FileUpload() instance
        
    Returns
    -------
    dict_dfs : dictionary
        Dictionary containing the filenames (keys from dict_files) as keys and Pandas dataframes 
        ('content' values from dict_files).
        
    See Also
    --------
    join_uploader : Puts functions create_uploader and create_button in a box
    create_uploader : Create file uploader widget.
    """
    dict_dfs = {}
    for key in uploader.value.keys():
        df_tmp = pd.read_csv(io.BytesIO(uploader.value[key]['content']))  
        dict_dfs[key] = df_tmp
    return dict_dfs


def create_toggle_buttons(options, tooltips=[]):
    """
    Create toggle buttons widget.

    It uses the ipywidget as widgets library to create a button instance.

    Parameters
    ----------
    options : list
        Text on toggle buttons
        
    tooltips : list
        Tooltips in toggle buttons

    Returns
    -------
    ipywidget.ToggleButtons instance
        ToggleButtons widget
    """
    toggle = widgets.ToggleButtons(options=options, tooltips=tooltips, disabled=False)
    return toggle

def create_dropdown(options, text, tooltips=[]):
    """
    Create dropdown widget.

    It uses the ipywidget as widgets library to create a dropdown instance.

    Parameters
    ----------
    options : list
        List of options
        
    text : str
        Description to include on the list.
        
    tooltips : list
        Tooltips 

    Returns
    -------
    ipywidget.Dropdown instance
        Dropdown widget
    """
    return widgets.Dropdown(options=options, description=text, disabled=False)

def create_combobox(options, text, tooltips=[]):
    """
    Create Combobox widget.

    It uses the ipywidget as widgets library to create a Combobox instance.

    Parameters
    ----------
    options : list
        List of options
        
    text : str
        Description to include on the widget.
        
    tooltips : list
        Tooltips 

    Returns
    -------
    ipywidget.Combobox instance
        Combobox widget
    """
    return widgets.Combobox(options=options, placeholder=text, ensure_option=True, disabled=False)

def create_text_area (text, tooltips=[]):
    """
    Create Textarea widget.

    It uses the ipywidget as widgets library to create a Textarea instance.

    Parameters
    ----------  
    text : str
        Description to include on the widget.
        
    tooltips : list
        Tooltips 

    Returns
    -------
    ipywidget.Textarea instance
        Textarea widget
    """
    return widgets.Textarea(placeholder=text, disabled=False, layout = widgets.Layout(width='95%'))

## Outputs

### Upload a file

#### Functions

In [6]:
def button_upload_file_action(b):
    """
    Show the name of the file that has been uploaded.
    
    Parameters
    ----------
    b : handler, callable

    Returns
    -------
    None
    
    See Also
    --------
    join_uploader : Puts functions create_uploader and create_button in a box
    create_button : Create button widget
    """
    global uploader_single, df, df_original, tab, tab_id 
    global title, after_upload, after_filter, after_function, after_plot
    
    dict_dfs = do_files_to_dfs (uploader_single)
    df = list(dict_dfs.values())[0]
    df_original = df.copy()
    
    #tab, dict_selection = do_filters_tab()
    output_upload.clear_output()

    with output_upload:
        print("File uploaded: {}".format(list(uploader_single.value.keys())))
        
    out.clear_output()
    with out:
        
        title = create_text_area("Include title of the report.")  
        display(title)
        
        after_upload = create_text_area("Add descriptions or comments about the dataset.")  
        display(after_upload)
            
        display("Filter tab")
        tab_id = 0
        tab, dict_selection = do_filters_tab()
        display (tab)
        tab_buttons (dict_selection, "Save all changes", "Reset all changes")
        display (output_tab)
        after_filter = create_text_area("Add descriptions or comments about the filtering.")
        display(after_filter)
        
        display("Table analysis")
        choose_agg_funcs ()
        display (output_functions)
        after_function = create_text_area("Add descriptions or comments about the last dataframe created.")
        display(after_function)
        
        display("Plots")
        do_charts_tab ()
        display (output_plots)  
        after_plot = create_text_area("Add descriptions or comments about the last plot created.")
        display(after_plot)

#### Functions

##### Actions/analysis

In [7]:
def filter_df (dict_sel):
    """
    Filters dataframe based on options selected before.
    
    Parameters
    ----------
    df : Dataframe
        Dataframe to be filtered.
    
    dict_selection : dictionary
        Dictionary whose keys are the names of columns filtered and its values, the values chosen

    Returns
    -------
    None
    """    
    global df
    for key in dict_sel.keys():
        if df[key].dtype == object and dict_sel[key] != None:
            df = df[df[key].isin(dict_sel[key])]
        elif ((df[key].dtype == int) or (df[key].dtype == float)) and dict_sel[key] != None:
            df = df[(df[key] >= dict_sel[key][0]) & (df[key] <= dict_sel[key][1])]

##### Filters by datatype

In [8]:
def do_filter_string (options, rows=3, name = 'Select options:'):
    """
    Creates a dropdown list of options, where multiple can be selected by pressing CTRL (or CMD).
    
    Parameters
    ----------
    options : list
        List of options available
    
    rows : int, default 3
        Number of rows visible per scroll
    
    name : string
        Description of widget

    Returns
    -------
    flt: ipywidget SelectMultiple instance
        SelectMultiple dropdown widget
    """
    flt = widgets.SelectMultiple(options = options, value = options, 
                                 rows=rows, disabled = False, description = name,
                                layout=widgets.Layout(width='70%'),
                                style = {'description_width': 'initial'})
    return flt #try accordion?

def do_filter_float (ranges, step=1, name = 'Select range:'):
    """
    Float slider widget between a range of values
    
    Parameters
    ----------
    ranges : list or array of size 2
        List or array containing the minimum and maximum values for the slider
    
    step : int, default 1
        Step of the slider
    
    name : string
        Description of widget

    Returns
    -------
    flt: ipywidget FloatRangeSlider instance
        FloatRangeSlider widget
    """    
    flt = widgets.FloatRangeSlider(values = ranges, min=ranges[0], max=ranges[1]+step, step=step, 
                           description = name, disabled=False,
                           continuous_update=False, orientation='horizontal',
                           readout=True, readout_format='.2f',
                            layout=widgets.Layout(width='70%'),
                            style = {'description_width': 'initial'}) 
    return flt 

def do_filter_int (ranges, step=1, name = 'Select range:'):
    """
    Integer slider widget between a range of values
    
    Parameters
    ----------
    ranges : list or array of size 2
        List or array containing the minimum and maximum values for the slider
    
    step : int, default 1
        Step of the slider
    
    name : string
        Description of widget

    Returns
    -------
    flt: ipywidget FloatRangeSlider instance
        FloatRangeSlider widget
    """  
    flt = widgets.IntRangeSlider(values = ranges, min=ranges[0], max=ranges[1], step=step, 
                           description = name, disabled=False,
                           continuous_update=False, orientation='horizontal',
                           readout=True, readout_format='d',
                            layout=widgets.Layout(width='70%'),
                            style = {'description_width': 'initial'}) #values is not working
    return flt 

def do_filter_date (df, col, step=1, name = 'Select ints:'): # predefined range?
    start_date = widgets.DatePicker(
      description='Start Date',
      disabled=False
  )
    end_date = widgets.DatePicker(
      description='End Date',
      disabled=False
  )
    flt = widgets.HBox([start_date, end_date]) 
    return flt

##### Filter actions

In [9]:
def filter_string_action (widget, use_col, dict_sel):
    """
    Filters dataframe displayed based on options selected.
    It changes global variable dict_selection.
    
    Parameters
    ----------
    widget : Selection ipywidget widget
        It can be any selection widget (except maybe sliders), including widgets that allow multiple selection.
    
    use_col : string
        Name of the dataframe's column to filter

    Returns
    -------
    None
    
    See Also
    --------
    do_filter_string : Creates a dropdown list of options
    """  
    global df, df_original
    selection = list(widget['new'])
    dict_sel[use_col] = selection
    df_tmp = df[df[use_col].isin(selection)]
    output_tab.clear_output()
    with output_tab:
        display(df_tmp)
        
def filter_range_action (widget, use_col, dict_sel):
    """
    Filters dataframe displayed based on range selected.
    It changes global variable dict_selection.
    
    Parameters
    ----------
    widget : Slider ipywidget widget
    
    use_col : string
        Name of the dataframe's column to filter

    Returns
    -------
    None
    
    See Also
    --------
    do_filter_int : Creates an  integer range slider widget 
    do_filter_float : Creates an float range slider widget     
    """  
    selection = list(widget['new'])
    dict_sel[use_col] = selection
    output_tab.clear_output()
    df_tmp = df[(df[use_col] >= selection[0]) & (df[use_col] <= selection[1])]
    with output_tab:
        display(df_tmp)

##### Save Buttons

In [10]:
def get_tab_id(widget):
    global tab_id
    if widget['name'] == 'selected_index':
        tab_id = widget['new']

def save_tab_button(b, dict_sel, all_cols):
    """
    Callable function that applies function filter_df() when the associated 
    widget (a button) changes state.
    
    Parameters
    ----------
    dict_sel : dictionary
        Dictionary where the keys are the names of the dataframe's columns 
        filtered and values are the values applied.
    Returns
    -------
    None
    
    See Also
    --------
    filter_df : Filters dataframe based on options selected before
    tab_buttons : Creates 'Save' and 'Reset' button ipywidgets
    """ 
    global df, tab_id, tab
    with output_tab:
        if all_cols == True:
            filter_df(dict_sel)
        elif all_cols == False:
            filter_df({tab.get_title(tab_id) : dict_sel[tab.get_title(tab_id)]})
        
def reset_tab_button(b, df_original, dict_sel, all_cols):
    """
    Callable function that resets the dataframe and dict_sel passed when the associated 
    widget (a button) changes state.
    
    Parameters
    ----------
    df_original : dataframe
        A copy of the dataframe initially uploaded
    dict_sel : dictionary
        Dictionary where the keys are the names of the dataframe's columns 
        filtered and values are the values applied.
    Returns
    -------
    None
    
    See Also
    --------
    tab_buttons : Creates 'Save' and 'Reset' button ipywidgets
    """ 
    global df, tab_id, tab
    output_tab.clear_output()
    with output_tab:
        df = df_original.copy()
        if all_cols == True:
            dict_sel = {v: None for v in dict_sel}
            display(df)
        elif all_cols == False:
            dict_sel[tab.get_title(tab_id)] = None
            filter_df(dict_sel)
            display(df)
            
def tab_buttons (dict_sel, save_text, reset_text):
    """
    Creates 'Save' and 'Reset' buttons ipywidgets.
    Executes functions save_tab_button() and reset_tab_button() on click
    
    Parameters
    ----------
    dict_sel : dictionary
        Dictionary where the keys are the names of the dataframe's columns 
        filtered and values are the values applied.
    Returns
    -------
    None
    
    See Also
    --------
    save_tab_button : Callable function that applies filter_df() when state changes
    reset_tab_button : Callable function that resets dict_sel and global variable df when state changes
    """ 
    global df, df_original
    
    button_save = create_button(save_text)
    button_reset = create_button(reset_text)
    
    display(widgets.HBox([button_save, button_reset]))

    button_save.on_click(functools.partial(save_tab_button, dict_sel = dict_sel, all_cols=True))
    button_reset.on_click(functools.partial(reset_tab_button, 
                                        df_original = df_original, dict_sel = dict_sel, all_cols=True))

##### Filter tab

In [11]:
def do_filters_tab ():
    """
    Creates a tab with filters as children.
    The filters can be of type 'object', 'int', or 'float'.
    It changes the global variable df.
    
    Parameters
    ----------
    None

    Returns
    -------
    tab: ipywidgets Tab instance
        ipywidgets Tab instance populated with filters as children
    dict_selection : dictionary
        Dictionary where the keys are the names of the dataframe's columns 
        filtered and values are the values applied
    
    See Also
    --------
    do_filter_string : Creates a dropdown list of options
    filter_string_action : Filters dataframe based on options selected
    do_filter_int : Creates an integer range slider widget 
    do_filter_float : Creates an float range slider widget  
    filter_range_action : Filters dataframe based on range selected
    """  
    global df, df_original
    cols = df.dtypes.index
    cols_types = df.dtypes.values
    children = []
    tab = widgets.Tab()
    for col, i in zip(cols, range(len(cols))):
        tab.set_title(i, col)

    dict_selection = dict(zip(cols, len(cols)*[None]))
    button_save = create_button("Save filter changes")
    button_reset = create_button("Reset filter changes")

    
    checkbox = widgets.Checkbox(value=False,description='Include null values',
                                disabled=False,indent=False) 

    for col, tp in zip(cols, cols_types):
        options = []
        if tp == object:
            options = list(df[col][df[col].notnull()].unique())
            flt = do_filter_string (options)
            flt.observe(functools.partial(filter_string_action, use_col=col,
                                         dict_sel = dict_selection), names='value')
        elif tp == int:
            options = [df[col].min(), df[col].max()]
            flt = do_filter_int (options)
            flt.observe(functools.partial(filter_range_action, use_col=col,
                                         dict_sel = dict_selection), names='value')  
        
        elif tp == float:
            options = [df[col].min(), df[col].max()]
            flt = do_filter_float (options)
            flt.observe(functools.partial(filter_range_action, dict_sel = dict_selection, 
                                          use_col=col), names='value')  
                
        flt = widgets.VBox([widgets.HBox([flt, button_save, button_reset]), checkbox])
        children.append(flt)
        
    tab.observe(get_tab_id)    
    button_save.on_click(functools.partial(save_tab_button, dict_sel = dict_selection, all_cols=False))   
    button_reset.on_click(functools.partial(reset_tab_button, 
                                        df_original = df_original, dict_sel = dict_selection, all_cols=False))
    tab.children = children

    return tab, dict_selection

### Do table analysis

#### Functions

In [12]:
def do_agg_funcs (widget, function):
    """
    Does predefined aggregated functions on global variable df.
    Last dataframe created is saved on global variable last_func.

    Parameters
    ----------
    function :  function, str, list or dict 
    Function to use for aggregating the data. If a function, must either work when passed a DataFrame or when passed to DataFrame.apply.
    Accepted combinations are:
        - function
        - string function name
        - list of functions and/or function names, e.g. [np.sum, 'mean']
        - dict of axis labels -> functions, function names or list of such.
        
    Returns
    -------
    None
    
    See Also
    --------
    choose_agg_funcs : Creates and displays selection widgets for aggregated functions
    """  
    global df, last_func
    selection = list(widget['new'])    
    output_functions.clear_output()
    with output_functions:
        last_func = df.groupby(selection).agg(function)
        display(last_func)
        
def choose_agg_funcs ():
    """
    Creates and displays selection widgets for aggregated functions.
    Observes values selected.

    Parameters
    ----------
    None      

    Returns
    -------
    None
    
    See Also
    --------
    do_agg_funcs : Does predefined aggregated functions on global variable df
    """  
    global df
    count_selection = do_filter_string (list(df.columns), rows=6, name = 'Count of')
    sum_selection = do_filter_string (list(df.columns), rows=6, name = 'Sum of')
    mean_selection = do_filter_string (list(df.columns), rows=6, name = 'Mean of')

    count_selection.observe(functools.partial(do_agg_funcs, function = 'count'), names='value')
    sum_selection.observe(functools.partial(do_agg_funcs, function = 'sum'), names='value')
    mean_selection.observe(functools.partial(do_agg_funcs, function = 'mean'), names='value')
    
    tab = widgets.Tab()
    tab.set_title(0, "Aggregated functions")
    #tab.set_title(1, "Complex charts")
    
    agg_funcs = widgets.HBox([count_selection, sum_selection, mean_selection])

    tab.children = [agg_funcs]
    
    display(tab)

### Do plots

#### Functions

In [13]:
def choose_option (widget, option):
    global options_basic
    selection = widget['new']    
    output_plots.clear_output()
    with output_plots:
        options_basic[option] = selection
        
def do_basic_chart (b, dict_options):
    global df, last_plot
    with output_plots:
        x = dict_options['x']
        y = dict_options['y']
        if dict_options['type'] == 'Scatter':
            last_plot = px.scatter(df, x=x, y=y)
        elif dict_options['type'] == 'Line':
            last_plot = px.line(df, x=x, y=y)           
        elif dict_options['type'] == 'Bar':
            last_plot = px.bar(df, x=x, y=y)
        display(last_plot)

def do_charts_tab (plots_types = ['Scatter', 'Line', 'Bar']):
    global df, options_basic
    options_basic = {'type': plots_types[0], 'x': list(df.columns)[0], 'y': list(df.columns)[0]}

    tp = create_combobox(plots_types, "Type of plot:")
    x = create_combobox(list(df.columns), "x:")
    y = create_combobox(list(df.columns), "y:")

    x.observe(functools.partial(choose_option, option = 'x'), names='value')
    y.observe(functools.partial(choose_option, option = 'y'), names='value')
    tp.observe(functools.partial(choose_option, option = 'type'), names='value')

    tab = widgets.Tab()
    tab.set_title(0, "Basic Charts")
    #tab.set_title(1, "Complex charts")
    
    plot_button = create_button('Plot it')
    plot_button.on_click(functools.partial(do_basic_chart, dict_options = options_basic))

    basic = widgets.HBox([tp, x, y, plot_button])

    tab.children = [basic]
    
    display(tab)

### Overall display

In [14]:
global df, df_original, tab, tab_id, last_func, last_plot
global title, after_upload, after_filter, after_function, after_plot


display("Upload section")

box, uploader_single, button_upload_file = join_uploader("Show uploaded files", True)
button_upload_file.on_click(button_upload_file_action)
output_upload = widgets.Output()
display(box, output_upload)

out = widgets.Output()
output_tab = widgets.Output()
output_functions = widgets.Output()
output_plots = widgets.Output()

display (out)

'Upload section'

Output()

Output()

In [15]:
a = We filtered the data to include only entire homes or apartments that require a minimum of nights between 1 and 7, have more than 10 reviews and more than 2 reviews per month.

SyntaxError: invalid syntax (3171438367.py, line 1)

In [33]:
df_html = df.loc[df.index[0]:df.index[10]].to_html()
last_func_html = last_func.to_html()
last_plot_img = last_plot.to_image(format="png")

html = """
<!DOCTYPE html>
<html>
<h1>{}</h1>

<body>

<h2>Dataset description</h2>
<p>{}</p>

<h2>Filtered dataset (first 10 rows)</h2>
<p>{}</p>

<h3>Filtered dataset description</h3>
<p>{}</p>

<h2>Aggregated results</h2>
<p>{}</p>

<h3>Aggregated results description</h3>
<p>{}</p>

<h2>Plot</h2>
<img src={} width="500" height="600"> 

<h3>Plot description</h3>
<p>{}</p>

</body>
</html> 
""".format(title.value, after_upload.value, df_html, after_filter.value, last_func_html, after_function.value, last_plot_img, after_plot.value)

In [34]:
file = open("sample.html","w")

file.write(html)

file.close()